In [1]:
#https://stackoverflow.com/questions/35215161/most-efficient-way-to-map-function-over-numpy-array

import timeit
import numpy as np

f = lambda x: x ** 2
vf = np.vectorize(f)
uf=np.frompyfunc(f, 1, 1)

def test_direct(x, n):
    t = timeit.timeit(
        'f(x)',
        'from __main__ import x, f', number=n)
    print('direct: {0:.3f}'.format(t))

def test_vectorized(x, n):
    t = timeit.timeit(
        'vf(x)',
        'from __main__ import x, vf', number=n)
    print('vectorized: {0:.3f}'.format(t))

def test_ufunc(x, n):
    t = timeit.timeit(
        'uf(x)',
        'from __main__ import x, uf', number=n)
    print('universal: {0:.3f}'.format(t))

In [2]:
x = np.arange(1000)
n = 10000
test_direct(x, n)      # 0.007
test_vectorized(x, n)  # 0.945
test_ufunc(x,n)


direct: 0.052
vectorized: 4.732
universal: 4.104


In [3]:
x = np.arange(1000)
n = 10000
t = timeit.timeit(
        'for i in range(len(x)): x[i]=f(x[i])',
        'from __main__ import np, x, f', number=n)
print('t: {0:.3f}'.format(t))

t: 5.232


In [4]:
L=4
base=np.arange(L*L,dtype=np.double).reshape([L,L])
print(base)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


In [5]:
"""ravel y unravel son cosas que se hacen tipicamente en lattices cuando se usa C"""
np.unravel_index(6,base.shape)

(1, 2)

In [6]:
np.unravel_index(6,(3,3,3))

(0, 2, 0)

In [7]:
np.ravel_multi_index((1,2),base.shape)

6

In [8]:
np.ravel_multi_index((2,4),base.shape,mode='wrap')

8

In [9]:
np.unravel_index(range(27),(3,3,3))

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2]),
 array([0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1,
        1, 1, 2, 2, 2]),
 array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
        1, 2, 0, 1, 2]))

In [10]:
[np.unravel_index(i,(3,3,3)) for i in range(27)]

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2)]

In [11]:
L=4
import numpy as np
base=(np.arange(L*L*L,dtype=np.int8)+65).reshape([L,L,L]).view('S1')    

In [12]:
base

array([[[b'A', b'B', b'C', b'D'],
        [b'E', b'F', b'G', b'H'],
        [b'I', b'J', b'K', b'L'],
        [b'M', b'N', b'O', b'P']],

       [[b'Q', b'R', b'S', b'T'],
        [b'U', b'V', b'W', b'X'],
        [b'Y', b'Z', b'[', b'\\'],
        [b']', b'^', b'_', b'`']],

       [[b'a', b'b', b'c', b'd'],
        [b'e', b'f', b'g', b'h'],
        [b'i', b'j', b'k', b'l'],
        [b'm', b'n', b'o', b'p']],

       [[b'q', b'r', b's', b't'],
        [b'u', b'v', b'w', b'x'],
        [b'y', b'z', b'{', b'|'],
        [b'}', b'~', b'\x7f', b'\x80']]], dtype='|S1')

In [13]:
zplus=np.transpose(np.unravel_index(range(L*L*L),(L,L,L)))+(0,0,1)
np.shape(zplus)

(64, 3)

In [14]:
zp=np.array([np.ravel_multi_index(i,(L,L,L),mode='wrap') for i in zplus])

In [15]:
base.take(zp).reshape((L,L,L))

array([[[b'B', b'C', b'D', b'A'],
        [b'F', b'G', b'H', b'E'],
        [b'J', b'K', b'L', b'I'],
        [b'N', b'O', b'P', b'M']],

       [[b'R', b'S', b'T', b'Q'],
        [b'V', b'W', b'X', b'U'],
        [b'Z', b'[', b'\\', b'Y'],
        [b'^', b'_', b'`', b']']],

       [[b'b', b'c', b'd', b'a'],
        [b'f', b'g', b'h', b'e'],
        [b'j', b'k', b'l', b'i'],
        [b'n', b'o', b'p', b'm']],

       [[b'r', b's', b't', b'q'],
        [b'v', b'w', b'x', b'u'],
        [b'z', b'{', b'|', b'y'],
        [b'~', b'\x7f', b'\x80', b'}']]], dtype='|S1')

In [16]:
np.char.add(base,base.take(zp).reshape((L,L,L)))

array([[[b'AB', b'BC', b'CD', b'DA'],
        [b'EF', b'FG', b'GH', b'HE'],
        [b'IJ', b'JK', b'KL', b'LI'],
        [b'MN', b'NO', b'OP', b'PM']],

       [[b'QR', b'RS', b'ST', b'TQ'],
        [b'UV', b'VW', b'WX', b'XU'],
        [b'YZ', b'Z[', b'[\\', b'\\Y'],
        [b']^', b'^_', b'_`', b'`]']],

       [[b'ab', b'bc', b'cd', b'da'],
        [b'ef', b'fg', b'gh', b'he'],
        [b'ij', b'jk', b'kl', b'li'],
        [b'mn', b'no', b'op', b'pm']],

       [[b'qr', b'rs', b'st', b'tq'],
        [b'uv', b'vw', b'wx', b'xu'],
        [b'yz', b'z{', b'{|', b'|y'],
        [b'}~', b'~\x7f', b'\x7f\x80', b'\x80}']]], dtype='|S2')